In [1]:
import pandas as pd
import string
import re 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import sklearn
import numpy as np

In [2]:
data = pd.read_csv('C:\\Users\\zeeshan lone\\Desktop\\datasets\\imdb.csv')
data_replica = pd.read_csv('C:\\Users\\zeeshan lone\\Desktop\\datasets\\replica.csv')
print(data.head())
data = data[0:25000]

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [3]:
print('\nThe shape of the dataset is : {}'.format(data.shape))


The shape of the dataset is : (25000, 2)


In [4]:
# remove punctuation from the data first

punctuations = string.punctuation
def remove_punctuations(data_1):
    text_non_pun = "".join([word for word in data_1 if word not in punctuations])
    return text_non_pun



In [5]:
data_replica['data_no_punctuation'] = data['review'].apply(lambda x:remove_punctuations(x))

In [6]:
# tokenise the data step 2

def tokenize(data_2):
    tokens = re.split('\W+',data_2)
    return tokens


In [7]:
data_replica['data_no_punctuation'] = data_replica['data_no_punctuation'].apply(lambda x:tokenize(x))

In [8]:
# step three is to remove the stop words from the data

StopWords = set(stopwords.words('english'))

def remove_stopwords(data_3):
    text_no_stopwords = [word for word in data_3 if word not in StopWords ]
    return text_no_stopwords
    

In [9]:
data_replica['data_no_punctuation'] = data_replica['data_no_punctuation'].apply(lambda x:remove_stopwords(x))


In [10]:
# step 4 is to lammatize or stemming the data 

w_n_lemmatizer = WordNetLemmatizer()

def lemmatize_data(data_4):
    lemmatized_text = [w_n_lemmatizer.lemmatize(word) for word in data_4]
    return lemmatized_text

In [11]:
data_replica['data_no_punctuation'] = data_replica['data_no_punctuation'].apply(lambda x:lemmatize_data(x))

In [12]:
# step 5 is to extract features from our data so that we can use it in machine learning
# since the data is in list of list format we will convert it to list of string first to be given as input to TFIDF
final_str = ''
def converttostr(input_seq, seperator):
   final_str = seperator.join(input_seq)
   return final_str


In [13]:
seperator=' '
abc=[]
for j in data_replica['data_no_punctuation']:
    abc.append(converttostr(j, seperator))
print(type(abc))


<class 'list'>


In [14]:
print(data_replica['data_no_punctuation'].shape)

(25000,)


In [15]:
# TFIDF vectorization by converting all text to lower case and using n-grams
tfidf_vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(1, 3))
training_data_vectorized = tfidf_vectorizer.fit_transform(abc)
print(training_data_vectorized.shape)

(25000, 4862393)


In [16]:
y_label = data.drop(['review'], axis=1)
y_label = np.array(y_label)
y_label = y_label.reshape(-1)
y_label = sklearn.utils.validation.column_or_1d(y_label, warn=True)
print(y_label)

['positive' 'positive' 'positive' ... 'positive' 'positive' 'negative']


In [22]:
x_train = training_data_vectorized[:10000]
x_test =  training_data_vectorized[10000:11000]
y_train = y_label[:10000]
y_test = y_label[10000:11000]

In [19]:
sup_vect_clas = SVC(gamma='auto')
sup_vect_clas.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
print('accuracy of the model is {}'.format(sup_vect_clas.score(x_test,y_test)))

accuracy of the model is 0.466


In [25]:
yy_test = y_test[0:10]
print(yy_test)
xx_test = x_test[0:10]
print(xx_test)
sup_vect_clas.predict(xx_test)

['negative' 'positive' 'negative' 'positive' 'negative' 'negative'
 'negative' 'negative' 'negative' 'positive']
  (0, 21818)	0.040320064126252955
  (0, 21853)	0.06190443262268353
  (0, 21855)	0.06440741828823751
  (0, 22282)	0.03944916869752276
  (0, 22370)	0.06440741828823751
  (0, 22371)	0.06440741828823751
  (0, 38072)	0.04163551458810298
  (0, 38073)	0.06440741828823751
  (0, 38074)	0.06440741828823751
  (0, 91648)	0.018438738399887074
  (0, 94897)	0.06440741828823751
  (0, 94898)	0.06440741828823751
  (0, 105343)	0.050509918877787455
  (0, 105354)	0.06440741828823751
  (0, 105355)	0.06440741828823751
  (0, 127649)	0.0265051605721952
  (0, 128447)	0.06440741828823751
  (0, 128448)	0.06440741828823751
  (0, 240017)	0.020777358363745623
  (0, 240504)	0.04111770393606265
  (0, 240516)	0.06440741828823751
  (0, 243299)	0.020348126225146906
  (0, 244705)	0.06440741828823751
  (0, 244706)	0.06440741828823751
  (0, 465263)	0.019976770225545636
  :	:
  (9, 4732576)	0.027282872058876518
  

array(['positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive'],
      dtype=object)